In [24]:

from tarski.theories import Theory
from tarski.io.fstrips import ParsingError, FstripsReader
from tarski.syntax import *

# Parsing and Processing PDDL

In [25]:
reader = FstripsReader(raise_on_error=True)

In [26]:
reader.parse_domain('probBLOCKS-domain.pddl')
reader.parse_instance('probBLOCKS-4-2.pddl')

In [27]:
L = reader.problem.language

We can retrieve any sort (or _type_ in PDDL-parlance), constant, function or predicate 
via the `get` method of the class `Language`. Since we are dealing
with the classic `Blocks World` benchmark we know there must be predicates `on(x,y)` and `clear(x)` 

In [28]:
on = L.get('on')
clear = L.get('clear')

`Language` instances allow us to iterate over the list of declared predicates etc. via the
properties `predicates`, `functions`

In [29]:
for pred in L.predicates:
    print(str(pred))

=/2
!=/2
on/2
ontable/1
clear/1
handempty/0
holding/1


As seen above `Tarski` parser defines the equality predicate and its negation, as  well as the
usual `on` etc. The number after the name of the predicate indicates its _arity_ (i.e. number of arguments).

Getting our hands on the objects defined in the instance file `probBLOCKS-4-2.pddl` requires to 
navigate the type hierarchy. First, we can take a look at what are the sorts defined similarly as
we do for predicates

In [30]:
for S in L.sorts:
    print(str(S))

Sort(object)


There is only one sort defined, `object`. We can check out its elements 

In [31]:
objects = L.get('object')
for obj in objects.domain():
    print(str(obj), type(obj))

c <class 'tarski.syntax.terms.Constant'>
b <class 'tarski.syntax.terms.Constant'>
d <class 'tarski.syntax.terms.Constant'>
a <class 'tarski.syntax.terms.Constant'>


### Excursion: Indexing Logical Elements

It is handy to have the objects indexed in a table for easy retrieval. We can have such a table
by name

In [32]:
obj_table1 = { str(obj): obj for obj in objects.domain()}
obj_table1

{'a': a (object), 'b': b (object), 'c': c (object), 'd': d (object)}

Alternatively, we can use the constants themselves as keys in a dictionary, by wrapping them with
an adapter in this way

In [34]:
obj_table2 = { symref(obj): [] for obj in objects.domain()}
obj_table2

{symref[a]: [], symref[b]: [], symref[c]: [], symref[d]: []}

Wrapping constant, terms and atoms is necessary as `Tarski` overloads most of the standard Python 
operators to provide compact syntax for formulas and expressions. For instance, if we wanted
to compare whether two constants are the same _value_ one could try the following

In [35]:
x = L.get('a')
y = L.get('b')
x == y

=(a,b)

Yet using the comparison operator `==` results in the _atom_ `=(a,b)`, as `a` and `b` stand for
two elements of a first-order language. If we use the wrappers

In [36]:
x = L.get('a')
y = L.get('b')
symref(x) == symref(y)

False

Then we get to compare the actual _physical_ objects than the _logical elements_ (e.g.
 constants, arithmetic expressions, Boolean formulas) they represent.

We can formulate a simple state constraint as we can do with any other `Tarski` construct